# This Notebook is for testing models that have been trained

In [1]:
#these need the necessary pip install instructions to run
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [139]:
#take the directory that contains the testing data set
data_dir_testing = "/Users/aaryangulia/Downloads/spanish_sl"

# Define the image size and batch size for the testing data set 
img_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data loading and augmentation
#innovatively augmented the data to be able to make most out of the least epochs and efficiently pass in data points
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2, 
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    fill_mode='nearest'  
)
#generate the testing data set
testing_generator = datagen.flow_from_directory(
    data_dir_testing,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
)

Found 1998 images belonging to 19 classes.


In [131]:
# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size) 
    img_array = image.img_to_array(img)  
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0 
    return img_array

# Load and preprocess the image
image_path = '/Users/aaryangulia/Downloads/indian_sl/5/46.jpg'
processed_image = load_and_preprocess_image(image_path)

In [142]:
model = tf.keras.models.load_model('epoch_custom_model.h5',custom_objects={'KerasLayer': hub.KerasLayer})

In [133]:
print("\n Model summary: ")
print(model.summary())


 Model summary: 
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 1001)              3540265   
                                                                 
 dense_3 (Dense)             (None, 36)                36072     
                                                                 
 activation_3 (Activation)   (None, 36)                0         
                                                                 
Total params: 3576337 (13.64 MB)
Trainable params: 36072 (140.91 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________
None


In [140]:
# Make a prediction using the pre-trained model
predictions = model.predict(processed_image)

# Get the predicted class label
predicted_class_idx = np.argmax(predictions[0])

# Decode the class index to get the class label (assuming you have a list of class labels)
class_labels = list(testing_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_idx]

# Get the confidence score of the prediction
confidence = predictions[0][predicted_class_idx]

# Print the results
print(f'Predicted class: {predicted_class_label}')
print(f'Confidence: {confidence:.2f}')

1/1 [==============================] - 0s 491ms/step
Predicted class: E
Confidence: 0.72


In [107]:
evaluation = model.evaluate_generator(testing_generator)
print(f'Test Loss: {evaluation[0]:.4f}')
print(f'Test Accuracy: {evaluation[1]*100:.2f}%')

<ipython-input-107-98e7fa83ce57>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation = model.evaluate_generator(testing_generator)


Test Loss: 2.2418
Test Accuracy: 28.33%


In [100]:
final_layer_weights = model.layers[-1].get_weights()
second_last_layer_weights = model.layers[-2].get_weights()

# Modify the weights as desired and then go back and test the model. re-iterate through the process to get optimal weights for custom layers
final_layer_weights = [weight * 1.5 for weight in final_layer_weights]
modified_second_last_layer_weights = [weight * 1.2 for weight in second_last_layer_weights]

# Set the modified weights back to the layers
model.layers[-1].set_weights(final_layer_weights)
model.layers[-2].set_weights(modified_second_last_layer_weights)

In [143]:
#save the data set if it works well. this format is the best to convert to TensorFlowJs!
tf.keras.models.save_model(model, 'arabic_sl_model')

INFO:tensorflow:Assets written to: arabic_sl_model/assets


INFO:tensorflow:Assets written to: arabic_sl_model/assets
